In [ ]:
import sys
import os

# add parent directory to path
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

from ideal_genom_qc.UMAPplot import UMAPplot

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create interactive widgets for input
input_path = widgets.Text(
    value='',
    description='Path to input plink1.9 files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

input_name = widgets.Text(
    value='',
    description='Name of the plink1.9 files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

dependables_path = widgets.Text(
    value='',
    description='Path to dependable files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

output_path = widgets.Text(
    value='',
    description='Path to output files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
# Display the widgets
display(input_path, input_name, dependables_path, output_path)

# Function to get the text parameter values
def get_params():
    return input_path.value, input_name.value, dependables_path.value, output_path.value

In [ ]:
# Use the parameter values
path_params = get_params()
print(f"Input Path: {path_params[0]}")
print(f"Input Name: {path_params[1]}")
print(f"Dependables: {path_params[2]}")
print(f"Output Path: {path_params[3]}")


In [ ]:
# Create interactive widgets for list input
umap_n_neighbors = widgets.Textarea(
    value='5',
    description='UMAP n_neighbors (comma-separated):',
    style={'description_width': 'auto'},
    layout=widgets.Layout(width='50%')
)

umap_min_dist = widgets.Textarea(
    value='0.5',
    description='UMAP min_dist (comma-separated):',
    style={'description_width': 'auto'},
    layout=widgets.Layout(width='50%')
)

umap_metric = widgets.Textarea(
    value='euclidean',
    description='UMAP metric (comma-separated):',
    style={'description_width': 'auto'},
    layout=widgets.Layout(width='50%')
)

umap_pca = widgets.IntText(
    value=10,  # Default value
    description='Number PC for UMAP (integer):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

umap_maf = widgets.FloatText(
    value=0.01,  # Default value
    description='MAF (float):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

umap_mind = widgets.FloatText(
    value=0.2,  # Default value
    description='mind (float):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

umap_geno = widgets.FloatText(
    value=0.1,  # Default value
    description='geno (float):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

umap_ind_par = widgets.Textarea(
    value='50, 5, 0.2',
    description='indep pairwise (comma-separated):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

umap_hwe = widgets.FloatText(
    value=5e-8,  # Default value
    description='geno (float):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Display the widgets
display(umap_n_neighbors, umap_min_dist, umap_metric, umap_pca, umap_maf, umap_mind, umap_geno, umap_ind_par, umap_hwe)

# Function to get the list parameter values
def get_umap_params()->dict:

    umap_params = {}

    indep = umap_ind_par.value.split(',')
    
    umap_params['n_neighbors'] = [int(x.strip()) for x in umap_n_neighbors.value.split(',')]
    umap_params['min_dist'] = [float(x.strip()) for x in umap_min_dist.value.split(',')]
    umap_params['metric'] = umap_metric.value.split(',')
    umap_params['pca'] = umap_pca.value
    umap_params['maf'] = umap_maf.value
    umap_params['mind'] = umap_mind.value
    umap_params['geno'] = umap_geno.value
    umap_params['ind_pair'] = [int(indep[0]), int(indep[1]), float(indep[2])]
    umap_params['hwe'] = umap_hwe.value

    return umap_params

In [ ]:
umap_params = get_umap_params()
umap_params

In [ ]:
umap_plots = UMAPplot(
    input_path      =path_params[0], 
    input_name      =path_params[1], 
    dependables_path=path_params[2],
    output_path     =path_params[3],
    compute_all     =True
)

umap_steps = {
    'ld_pruning': (umap_plots.ld_pruning, (umap_params['maf'], umap_params['geno'], umap_params['mind'], umap_params['hwe'], umap_params['ind_pair'],)),
    'comp_pca'  : (umap_plots.compute_pcas, (umap_params['pca'],)),
    'draw_plots': (umap_plots.generate_plots, (umap_params['n_neighbors'], umap_params['min_dist'], umap_params['metric'], ))
}

umap_step_description = {
    'ld_pruning': 'LD pruning',
    'comp_pca'  : 'Compute PCAs',
    'draw_plots': 'Generate UMAP plots'
}

for name, (func, params) in umap_steps.items():
    print(f"\033[34m{umap_step_description[name]}.\033[0m")
    func(*params)